In [4]:
import operator
import functools
import multipledispatch
import dask
import dask.array as da

In [5]:
import sys
import os
PROJECT_PATH = os.path.realpath(os.path.join(os.getcwd(), '..'))
print PROJECT_PATH
sys.path.append(PROJECT_PATH)

/Users/Baris/Documents/Thesis/modules/scs-dask


In [6]:
from scs_dask.linalg import linear_operator as linop
from scs_dask.linalg import atoms

In [7]:
namespace_preconditioner = dict()
dispatch = functools.partial(
        multipledispatch.dispatch, namespace=namespace_preconditioner)

In [17]:
@dispatch(da.Array)
def jacobi_preconditioner(array, name=None):
    name = 'jacobi-precond-' + array.name if name is None else name
    m, n = array.shape
    assert m == n, 'preconditioner expects square linear operator'
    diag = da.diag(array)
    return linop.DLODiagonal(da.core.map_blocks(da.reciprocal, diag, name=name))

In [18]:
@dispatch(linop.DLODense)
def jacobi_preconditioner(linear_op, name=None):
    return jacobi_preconditioner(linear_op.data, name=name)

In [31]:
@dispatch(linop.DLOGram)
def jacobi_preconditioner(linear_op, name=None):
    name = 'jacobi-precond-' + linear_op.name if name is None else name
    diag = atoms.diag_gram(linear_op)
    return linop.DLODiagonal(da.core.map_blocks(da.reciprocal, diag, name=name))

In [32]:
import numpy as np
d_test = np.random.random(100)
A_test = da.from_array(np.diag(d_test), chunks=10)

In [33]:
(ones,) = dask.compute(jacobi_preconditioner(A_test).data * d_test)
assert np.linalg.norm(ones - 1) < 1e-15 * (1 + np.linalg.norm(ones))

In [34]:
(ones,) = dask.compute(jacobi_preconditioner(linop.DLODense(A_test)).data * d_test)
assert np.linalg.norm(ones - 1) < 1e-15 * (1 + np.linalg.norm(ones))

In [35]:
(ones,) = dask.compute(jacobi_preconditioner(linop.DLOGram(A_test)).data * d_test * d_test)
assert np.linalg.norm(ones - 1) < 1e-15 * (1 + np.linalg.norm(ones))

In [36]:
(ones,) = dask.compute(jacobi_preconditioner(linop.DLORegularizedGram(A_test)).data * (1 + d_test * d_test))
assert np.linalg.norm(ones - 1) < 1e-15 * (1 + np.linalg.norm(ones))

In [37]:
mu = 1 + np.random.random()
muIAA = mu + d_test * d_test
(ones,) = dask.compute(jacobi_preconditioner(linop.DLORegularizedGram(A_test, regularization=mu)).data * muIAA)
assert np.linalg.norm(ones - 1) < 1e-15 * (1 + np.linalg.norm(ones))